In [1]:
import pymc as pm
import aesara.tensor as at
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import default_rng

RANDOM_SEED = 31415
rng = default_rng(RANDOM_SEED)

In [2]:
n = 160
k_true = 4
d = 60
d_e = 1

In [3]:
Y = pd.read_csv('data/features/eda/2.csv').to_numpy()
Y = Y.T

df_ = pd.read_csv('data/LookAtMe_002.csv', sep='\t')
label = np.array(list([int(d>2) for d in df_['rating']]))
E = label[:,np.newaxis]
E = np.transpose(E)

array([[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 1]])

In [4]:
k = 3
coords = {"latent_columns": np.arange(k),
          "rows": np.arange(n),
          "observed_columns": np.arange(d),
          "observed_label":np.arange(d_e)}

In [5]:
def expand_packed_block_triangular(d, k, packed, diag=None, mtype="aesara"):
    # like expand_packed_triangular, but with d > k.
    assert mtype in {"aesara", "numpy"}
    assert d >= k

    def set_(M, i_, v_):
        if mtype == "aesara":
            return at.set_subtensor(M[i_], v_)
        M[i_] = v_
        return M

    out = at.zeros((d, k), dtype=float) if mtype == "aesara" else np.zeros((d, k), dtype=float)
    if diag is None:
        idxs = np.tril_indices(d, m=k)
        out = set_(out, idxs, packed)
    else:
        idxs = np.tril_indices(d, k=-1, m=k)
        out = set_(out, idxs, packed)
        idxs = (np.arange(k), np.arange(k))
        out = set_(out, idxs, diag)
    return out

In [6]:
def makeW(d, k, dim_names,name):
    # make a W matrix adapted to the data shape
    n_od = int(k * d - k * (k - 1) / 2 - k)
    # trick: the cumulative sum of z will be positive increasing
    z = pm.HalfNormal("W_z_"+name, 1.0, dims="latent_columns")
    b = pm.HalfNormal("W_b_"+name, 1.0, shape=(n_od,), dims="packed_dim")
    L = expand_packed_block_triangular(d, k, b, at.ones(k))
    W = pm.Deterministic(name, at.dot(L, at.diag(at.extra_ops.cumsum(z))), dims=dim_names)
    return W


In [7]:
with pm.Model(coords=coords) as PPCA_identified:
    W_eda = makeW(d, k, ("observed_columns", "latent_columns"),'W_eda')
    W_hr = makeW(d, k, ("observed_columns", "latent_columns"),'W_hr')
    W_e = pm.Normal("W_e", dims=("observed_label", "latent_columns"))
    C = pm.Normal("C", dims=("latent_columns", "rows"))
    psi = pm.HalfNormal("psi", 1.0)
    X_eda = pm.Normal("X_eda", mu=at.dot(W_eda, C), sigma=psi, observed=Y, dims=("observed_columns", "rows"))
    X_e = pm.Bernoulli("X_e", p=pm.math.sigmoid(at.dot(W_e, C)), dims=("observed_label", "rows"), observed=E)

In [8]:
gv = pm.model_to_graphviz(PPCA_identified)
gv.view('PPCA example')

'PPCA example.pdf'

In [9]:
with PPCA_identified:
    trace = pm.sample(300)

Only 300 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [W_z_W_eda, W_b_W_eda, W_e, C, psi]


Sampling 2 chains for 1_000 tune and 300 draw iterations (2_000 + 600 draws total) took 127 seconds.
There were 21 divergences after tuning. Increase `target_accept` or reparameterize.
There were 27 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6426, but should be close to 0.8. Try to increase the number of tuning steps.
